In [0]:


from pyspark.sql import types as T
from pyspark.sql import functions as F
from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view

spark.sparkContext.addPyFile("s3://b2c-prod-dca-bdp-data/BDP-PROD-APP-INT-QA/bdp/user_data/application_name=pipeline-production-data-ready-smoke/application_code/version=latest/code/bdp_resources/python/dependencies.zip")

class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass


import aaplproxy
def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()
sql_text ="""
WITH domain_id_name_mapping AS (
    SELECT
        db_table(
            engine="plproxy",
            database="aa",
            sql="SELECT domain_id AS domain_id, name AS domain_name FROM aa_domain_metadata WHERE is_disabled "=" 'f' AND is_disabled IN ('f') and sensitive_status IN (0)"
        )
);
"""


test_date='2020-03-24'
namespace = "aa.store.market-size.v1"
ingest_msg = {
    "namespace": "aa.store.market-size.v1",
    "job_type": "routine",
    "options":{},
    "source": [
        {}
    ]
}
run(spark, ingest_msg, sql_text)
spark.sql("select * from domain_id_name_mapping limit 10").show()


In [0]:
%md

1. db_table operator "=" need to add double quotes between equal
```WITH dna_universal_app_mapping AS (
    SELECT
        db_table(
            engine = "PG",
            database = "dna",
            schema = "universal_app_id int8, app_id int8, app_name text, market text, matched_rule text, comments text,
                      created_by int4, created_time text, last_updated_by int4, last_updated_time text, confirmed_by int4,
                      confirmed bool, disabled bool, automation_type int4, automation_id int2",
            sql = "SELECT * FROM dna_universal_app_mapping WHERE confirmed"
        )
);```


In [0]:
%md
